In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-
import os

import numpy as np
import torch
import torch.backends.cudnn as cudnn
from torch import nn

from dataloader import cifar10
from models import VGG11
from src import freeze_influence, hessians, selection, utils

device = "cuda" if torch.cuda.is_available() else "cpu"
target_removal_label = 0

In [3]:
def load_net(net, path):
    assert os.path.isfile(path), "Error: no checkpoint file found!"
    checkpoint = torch.load(path)
    net.load_state_dict(checkpoint["net"])
    return net


def save_net(net, path):
    dir, filename = os.path.split(path)
    if not os.path.isdir(dir):
        os.makedirs(dir)

    state = {
        "net": net.state_dict(),
    }
    torch.save(state, path)


def test(net, dataloader, criterion, label, include):
    net.eval()
    with torch.no_grad():
        net_loss = 0
        correct = 0
        total = 0
        num_data = 0
        for _, (inputs, targets) in enumerate(dataloader):
            if include:
                idx = targets == label
            else:
                idx = targets != label
            inputs = inputs[idx]
            targets = targets[idx]
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            net_loss += loss * len(inputs)
            num_data +=  len(inputs)

            total += targets.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()

        accuracy = correct / total * 100
        net_loss /= num_data
        return net_loss, accuracy


def get_full_param_list(net):
    """
    Return a list of parameter indices in flatten network.
    Warning: this function only provides indices of params when the param i) has requires_grad=True and 2) belongs to nn.Linear or nn.Conv2d
    """

    index_list = np.array([], dtype=int)
    start_index = 0
    for module in net.modules():
        if not list(module.children()) == []:
            continue

        num_param = sum(p.numel() for p in module.parameters() if p.requires_grad)
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            module_index_list = np.arange(num_param, dtype=int) + start_index
            index_list = np.append(index_list, module_index_list)

        start_index += num_param

    return index_list


def projected_influence(net, index_list, total_loss, target_loss, tol, step):
    full_param_list = get_full_param_list(net)
    influence = hessians.partial_influence(
        full_param_list, target_loss, total_loss, net, tol=tol, step=step
    )
    idx = np.isin(full_param_list, index_list)
    return influence[idx], full_param_list[idx]

In [4]:
net = VGG11().to(device)
net_name = "VGG11"

if device == "cuda":
    cudnn.benchmark = True

net_path = f"checkpoints/tab2/{net_name}/cross_entropy/ckpt_0.0.pth"
net = load_net(net, net_path)

net.eval()
num_param = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(
    f"==> Building {net_name} finished. "
    + f"\n    Number of parameters: {num_param}"
)

criterion = nn.CrossEntropyLoss()

# Data
print("==> Preparing data..")
batch_size = 512
num_workers = 16
num_sample_batch = 1
num_target_sample = 1024

data_loader = cifar10.CIFAR10DataLoader(batch_size, num_workers, validation=False)
train_loader, test_loader = data_loader.get_data_loaders()

loss, acc = test(net, test_loader, criterion, 11, False)
print(
    f"Original loss and acc : {loss:.4f}, {acc:.2f}%"
)

==> Building VGG11 finished. 
    Number of parameters: 9231114
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Original loss and acc : 0.3518, 91.50%


In [5]:
removal_inputs = list()
removal_targets = list()
for batch_idx, (inputs, targets) in enumerate(train_loader):
    idx = targets == target_removal_label
    removal_inputs.append(inputs[idx])
    removal_targets.append(targets[idx])
removal_inputs = torch.cat(removal_inputs)
removal_targets = torch.cat(removal_targets)

ratio_list = [.1, .3, .5]
result_list_GIF = []
result_list_FIF = []
result_list_PIF = []

tol = 1e-9

for param_ratio in ratio_list:
    for i in range(3):
        if i == 0:
            if_name = "GIF"
        elif i == 1:
            if_name = "FIF"
        else:
            if_name = "PIF"

        print(f"{if_name} - ratio: {param_ratio*100}%, tol: {tol}")
        for _ in range(10):
            # Initialize network
            net = load_net(net, net_path)

            # Compute total loss
            total_loss = 0
            for batch_idx, (inputs, targets) in enumerate(train_loader):
                if batch_idx >= num_sample_batch:
                    break
                idx = targets != target_removal_label
                inputs, targets = inputs[idx], targets[idx]
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                total_loss += criterion(outputs, targets)

            # Sampling the target removal data
            sample_idx = np.random.choice(len(removal_inputs), num_target_sample, replace=False)
            sample_removal_inputs = removal_inputs[sample_idx]
            sample_removal_targets = removal_targets[sample_idx]
            
            # Make hooks
            net_parser = selection.TopNActivations(net, param_ratio)
            net_parser.register_hooks()

            # Compute target loss
            target_loss = (
                criterion(net(sample_removal_inputs.to(device)), sample_removal_targets.to(device))
                * len(removal_inputs)
                / (len(train_loader.dataset) - len(removal_inputs))
            )
            
            # Delete hooks
            index_list = net_parser.get_parameters()
            net_parser.remove_hooks()

            if i == 0:
                influence = hessians.partial_influence(
                    net, total_loss, target_loss, index_list, tol=tol, step=1, max_iter=30
                )
            elif i == 1:
                influence = freeze_influence.freeze_influence(
                    net, total_loss, target_loss, index_list, tol=tol, step=1, max_iter=30
                )
            else:
                influence, index_list = projected_influence(
                    net, total_loss, target_loss, index_list, tol=tol, step=5, max_iter=30
                )

            scale = 10 if i != 2 else 60
            last_score = -1
            while True:
                utils.update_network(net, influence * scale, index_list)

                self_loss, self_acc = test(net, test_loader, criterion, target_removal_label, True)
                exclusive_loss, exclusive_acc = test(net, test_loader, criterion, target_removal_label, False)
                score = f1_score(self_acc, exclusive_acc)
                

                if last_score > score and score > .8:
                    print(
                    f"{scale:.2f} - test acc: {exclusive_acc:2.2f}, test loss: {exclusive_loss:.4f} | self-acc: {self_acc:2.2f}%, self loss: {self_loss:.4f} | Score: {score:.7f}"
                    )
                    break
                else:
                    if score < .7:
                        scale += .4 if i != 2 else 2.5
                    elif score > .85:
                        scale += .01 if i != 2 else 0.1
                    else:
                        scale += .1
                    if i == 0:
                        result_list_GIF += [exclusive_acc, exclusive_loss, self_acc, self_loss, score]
                    elif i == 1:
                        result_list_FIF += [exclusive_acc, exclusive_loss, self_acc, self_loss, score]
                    else:
                        result_list_PIF += [exclusive_acc, exclusive_loss, self_acc, self_loss, score]

                last_score = score
                
                if scale > 90:
                    break
        print("")

NameError: name 'target_removal_label' is not defined

In [ ]:
"""
retrained_net = VGG11().to(device)
net_name = retrained_net.__class__.__name__
net_path = f"../checkpoints/Figure_4/{net_name}/cross_entropy/ckpt_0.0_retrained.pth"
retrained_net = load_net(retrained_net, net_path)
flatten = False

loss, acc = test(retrained_net, test_loader, criterion, 11, False)
print(
    f"Original loss and acc : {loss:.4f}, {acc:.2f}%"
)
self_loss, self_acc = test(retrained_net, test_loader, criterion, 8, True)
exclusive_loss, exclusive_acc = test(retrained_net, test_loader, criterion, 8, False)
print(
    f"Retrained model \t Self: {self_loss:.2f} {self_acc:2.2f}% | Exclusive loss: {exclusive_loss:.2f}, {exclusive_acc:2.2f}%"
)
"""

## 